In [1]:
from osgeo import gdal
import os
from glob import glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [4]:
fo = '/home/geodata/Raster/modis/MYD16A2/raw/'
ff = sorted(glob(fo+'*.hdf'))
ff[:5]

['/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024001.h13v10.061.2024019001517.hdf',
 '/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024009.h13v10.061.2024027091412.hdf',
 '/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024017.h13v10.061.2024034011201.hdf',
 '/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024025.h13v10.061.2024041004244.hdf',
 '/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024033.h13v10.061.2024049000729.hdf']

In [5]:
ofo = '/home/geodata/Clientes/0FARMS/FabioRicardi-Barreiras_BA/modis/'
#extent = '-46.200579418 -11.200452249 -44.927499105 -12.210613802' # around the Bahia farm
extent = '-45.6861267089844034 -11.4570760726928995 -45.3921318054199006 -11.7891721725463992'

Path(ofo).mkdir( parents = True, exist_ok = True)


kwargs = {'format': 'GTiff', 'dstSRS': 'EPSG:4326'}
for f in ff:
    print(f)
    fn = f.split('.')[-5][1:]+'.tif'
    dataset = gdal.Open(f,gdal.GA_ReadOnly)

    # et and pet
    ds_et =  gdal.Open(dataset.GetSubDatasets()[0][0], gdal.GA_ReadOnly)
    ds_pet =  gdal.Open(dataset.GetSubDatasets()[2][0], gdal.GA_ReadOnly)

    # gdalwarp and clip
    ds = gdal.Warp(destNameOrDestDS=f'{ofo}Xet_{fn}',outputType=gdal.GDT_UInt16, srcDSOrSrcDSTab=ds_et, **kwargs)
    os.system(f'gdal_translate -projwin {extent} -of GTiff {ofo}Xet_{fn} {ofo}et_{fn}')
    os.system(f'rm {ofo}Xet_{fn}')
    del ds

    ds = gdal.Warp(destNameOrDestDS=f'{ofo}Xpet_{fn}',outputType=gdal.GDT_UInt16, srcDSOrSrcDSTab=ds_pet, **kwargs)
    os.system(f'gdal_translate -projwin {extent} -of GTiff {ofo}Xpet_{fn} {ofo}pet_{fn}')
    os.system(f'rm {ofo}Xpet_{fn}')
    del ds


/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024001.h13v10.061.2024019001517.hdf
Input file size is 3304, 2624
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 3304, 2624
0...10...20...30...40...50...60...70...80...90...100 - done.
/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024009.h13v10.061.2024027091412.hdf
Input file size is 3304, 2624
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 3304, 2624
0...10...20...30...40...50...60...70...80...90...100 - done.
/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024017.h13v10.061.2024034011201.hdf
Input file size is 3304, 2624
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 3304, 2624
0...10...20...30...40...50...60...70...80...90...100 - done.
/home/geodata/Raster/modis/MYD16A2/raw/MYD16A2.A2024025.h13v10.061.2024041004244.hdf
Input file size is 3304, 2624
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 3304

In [ ]:
def ArrayToTif(Array, Filename, Folder, Metadata, Type = 1):
    '''
    Converts a array like to .tif file.
    Array: Single or Multiband array
    Folder: Destination folder of files. Ex. /tmp/
    Filename: Name of output tif. Ex. MyTifFile
    Metadata: Dictionary with keys: GeoTrasform, projection, rows, cols of original
    raster.
    Type: 1 = gdal.GDT_Int16
          2 = gdal.GDT_Int32
          3 = gdal.GDT_Float32
          4 = gdal.GDT_Float64
    '''
    driver = gdal.GetDriverByName("GTiff")
    if not os.path.exists(Folder): os.makedirs(Folder)
    geot = Metadata['Geo']
    proj = Metadata['proj']
    rows = Metadata['rows']
    cols = Metadata['cols']
    if Type == 0:
        TypeIF = gdal.GDT_Byte
    if Type == 1:
        TypeIF = gdal.GDT_Int16
    if Type == 2:
        TypeIF = gdal.GDT_Int32
    if Type == 3:
        TypeIF = gdal.GDT_Float32
    if Type == 4:
        TypeIF = gdal.GDT_Float64
    if Type == 5:
        TypeIF = gdal.GDT_UInt16
    if Array.ndim == 2:
        Raster = driver.Create(Folder + '/' + Filename, cols, rows, 1, TypeIF)
        Raster.GetRasterBand(1).WriteArray(Array)
    else:
        Raster = driver.Create(Folder + '/' + Filename, cols, rows, Array.shape[2], TypeIF, [ '-co COMPRESS=DEFLATE -co PREDICTOR=1 -co ZLEVEL=9 -multi' ])
        for band in range(Array.shape[2]):
            print('Writing band' +str(band+1)+'of '+Filename)
            Raster.GetRasterBand(band+1).WriteArray(Array[:,:,band])
    Raster.SetGeoTransform(geot)
    Raster.SetProjection(proj)
    Raster.FlushCache()
    print('%s saved.\n '%(Filename))
    del geot, proj, rows, cols, Raster
    return


def TifToArray(Raster):
    '''
    This function opens a Raster data as Array to use numpy manipulation
    Return: Array, Metadata
    Array: A array like variable
    Metadata: Metadata of Raster file. Could be needed for a future\n
    ArrayToRaster transformation.
    '''

    openGdal = gdal.Open(Raster)
    geot = openGdal.GetGeoTransform()
    proj = openGdal.GetProjection()
    rows = openGdal.RasterYSize
    cols = openGdal.RasterXSize
    Metadata = {'Geo':geot, 'proj':proj, 'rows':rows, 'cols':cols}
    Array = openGdal.ReadAsArray()
    return Array, Metadata